<a href="https://colab.research.google.com/github/pranshulagrawal999/Movie_Review_Sentiment_Analysis/blob/main/Fine_tuning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Import necessary libraries
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the dataset
df_review = pd.read_csv("IMDB Dataset.csv")

# Prepare a balanced dataset
df_positive = df_review[df_review['sentiment'] == 'positive'][:9000]
df_negative = df_review[df_review['sentiment'] == 'negative'][:1000]
df_review_balanced = pd.concat([df_positive, df_negative])

# Convert sentiment labels to numerical format
df_review_balanced['label'] = df_review_balanced['sentiment'].map({'positive': 1, 'negative': 0})

# Split into training and testing sets
train_data, test_data = train_test_split(df_review_balanced, test_size=0.33, random_state=42)
train_dataset = Dataset.from_pandas(train_data[['review', 'label']])
test_dataset = Dataset.from_pandas(test_data[['review', 'label']])

# Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the data
def tokenize_data(example):
    return tokenizer(example['review'], truncation=True, padding='max_length', max_length=256)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

# Set the format for PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Define evaluation metrics
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10,
    save_total_limit=2
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)

# Test the model on new examples
examples = ["A good movie", "An excellent movie", "I did not like this movie at all I gave this movie away"]
encoded_examples = tokenizer(examples, truncation=True, padding=True, max_length=256, return_tensors="pt")
predictions = model(**encoded_examples)
predicted_classes = predictions.logits.argmax(-1).numpy()
print("Predictions:", predicted_classes)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.177000,0.158257,0.947576,0.958388,0.984965,0.971494
2,0.171400,0.183740,0.948485,0.963547,0.980287,0.971845


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.177000,0.158257,0.947576,0.958388,0.984965,0.971494
2,0.171400,0.183740,0.948485,0.963547,0.980287,0.971845
3,0.051200,0.213437,0.946061,0.962233,0.978951,0.970520


Evaluation Results: {'eval_loss': 0.2134370505809784, 'eval_accuracy': 0.946060606060606, 'eval_precision': 0.9622331691297209, 'eval_recall': 0.978950885399265, 'eval_f1': 0.9705200397482611, 'eval_runtime': 22.8059, 'eval_samples_per_second': 144.7, 'eval_steps_per_second': 2.28, 'epoch': 3.0}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import torch

# Check if CUDA (GPU) is available, otherwise fallback to CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Move the model to the appropriate device
model = model.to(device)

# Test the model on new examples
examples = ["A good movie", "An excellent movie", "I did not like this movie at all I gave this movie away"]

# Tokenize and move the inputs to the same device as the model
encoded_examples = tokenizer(examples, truncation=True, padding=True, max_length=256, return_tensors="pt")
encoded_examples = {key: value.to(device) for key, value in encoded_examples.items()}

# Make predictions
predictions = model(**encoded_examples)
predicted_classes = predictions.logits.argmax(-1).cpu().numpy()  # Move predictions back to CPU for further processing

print("Predictions:", predicted_classes)


Predictions: [1 1 0]


In [ ]:
!zip -r results.zip results/

  adding: results/ (stored 0%)
  adding: results/checkpoint-1257/ (stored 0%)
  adding: results/checkpoint-1257/tokenizer.json (deflated 71%)
  adding: results/checkpoint-1257/scheduler.pt (deflated 57%)
  adding: results/checkpoint-1257/model.safetensors (deflated 8%)
  adding: results/checkpoint-1257/rng_state.pth (deflated 25%)
  adding: results/checkpoint-1257/special_tokens_map.json (deflated 42%)
  adding: results/checkpoint-1257/config.json (deflated 46%)
  adding: results/checkpoint-1257/vocab.txt (deflated 53%)
  adding: results/checkpoint-1257/optimizer.pt (deflated 17%)
  adding: results/checkpoint-1257/tokenizer_config.json (deflated 76%)
  adding: results/checkpoint-1257/trainer_state.json (deflated 78%)
  adding: results/checkpoint-1257/training_args.bin (deflated 51%)
  adding: results/checkpoint-1250/ (stored 0%)
  adding: results/checkpoint-1250/tokenizer.json (deflated 71%)
  adding: results/checkpoint-1250/scheduler.pt (deflated 55%)
  adding: results/checkpoint-1250